## 检索机器人

In [1]:
from transformers import AutoTokenizer
import faiss
import pandas as pd

In [2]:
data = pd.read_csv("law_faq.csv")
data.head()

,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。


In [3]:
from dual import DualModel

In [5]:
dula_model = DualModel.from_pretrained("./dual_model/checkpoint-500")
dula_model.eval()

DualModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [6]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

## 转向量

In [13]:
import torch
from tqdm import tqdm

questions = data["title"].tolist()
vectors = []

with torch.inference_mode():
    for i in tqdm(range(0,len(questions),32)):
        batch_questions = questions[i:i+32]
        inputs = tokenizer(batch_questions,return_tensors= "pt", padding= True, max_length= 128, truncation= True)
        vector = dula_model.bert(**inputs)[1]
        vectors.append(vector)
    
vectors = torch.concat(vectors,dim=0).cpu().numpy()
vectors.shape


100%|██████████| 570/570 [08:34<00:00,  1.11it/s]


(18213, 768)

## 索引创建

In [14]:
index = faiss.IndexFlatIP(768)
faiss.normalize_L2(vectors)
index.add(vectors)
index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x0000025B53493330> >

In [19]:
## 对问题进行向量编码

question = "贷款"
with torch.inference_mode():
    inputs = tokenizer(batch_questions,return_tensors= "pt", padding= True, max_length= 128, truncation= True)
    vector = dula_model.bert(**inputs)[1]
    q_vector = vector.cpu().numpy()
q_vector.shape

(5, 768)

## 匹配

In [23]:
faiss.normalize_L2(q_vector)
score, id= index.search(q_vector,10)
topk_result = data.values[id[0].tolist()]
topk_result

array([['在一个APP贷款，3500元，18期还款，算下来需要换6000元，请问贷款方是否违法？',
        '18期就是18个月还6000那么，按1年12个月算的话，就是4000每个月，需要还款333.33，本金是3500/12=291.67每个月的利息就是114.3%，理论上，属于高利贷。可是，你如果确认贷款，你想钻他们的空子，比较难，建议你不用就是，要不然，后续的麻烦事，多了去了。'],
       ['钱站借贷2W5,分24期,每个月还约1660,这样算高利贷吗,可以要求减少不合理的部分金额吗',
        '民间借贷属于民事行为，受到民法和合同法的约束和保护。但根据《合同法》第二百一十一条规定：“自然人之间的借款合同约定支付利息的，借款的利率不得违反国家有关限制借款利率的规定”。依据《最高人民法院关于审理民间借贷案件适用法律若干问题的规定》，借贷双方约定的利率未超过年利率24%，出借人有权请求借款人按照约定的利率支付利息；但如果借贷双方约定的利率超过年利率36%，则超过年利率36%部分的利息应当被认定无效。《最高人民法院关于审理民间借贷案件适用法律若干问题的规定》第二十六条借贷双方约定的利率未超过年利率24%，出借人请求借款人按照约定的利率支付利息的，人民法院应予支持。借贷双方约定的利率超过年利率36%，超过部分的利息约定无效。借款人请求出借人返还已支付的超过年利率36%部分的利息的，人民法院应予支持。'],
       ['婚内男方个人名义的信用贷款，女方需要承担债务吗',
        '，婚内共同生活产生的债务一般为共同债务，如能证明该债务您不知情且未用于家庭共同生活，不须归还'],
       ['买单侠现金贷借25#00元，分期一年算下要还款3万七千多，请问这是高利贷吗？',
        '依据《最高人民法院关于审理民间借贷案件适用法律若干问题的规定》，借贷双方约定的利率未超过年利率24%，出借人有权请求借款人按照约定的利率支付利息；但如果借贷双方约定的利率超过年利率36%，则超过年利率36%部分的利息应当被认定无效。《最高人民法院关于审理民间借贷案件适用法律若干问题的规定》第二十六条借贷双方约定的利率未超过年利率24%，出借人请求借款人按照约定的利率支付利息的，人民法院应予支持。借贷双方约定的利率超过年利率36

## 加载交互模型,在交互中批量将topk问题与当前问题匹配，取argmax，再定位到相应答案

In [ ]:
from transformers import BertForSequenceClassification
cross_model = BertForSequenceClassification.from_pretrained("./cross_model/")